In [ ]:
import os, sys

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from nbafuns import *

player_dict = get_players_pbp()
teams_dict, teams_list = get_teams()

box_DIR = "../data/box/"
img_DIR_P = "../data/images/players/"
fig_DIR = "../figs/players/"

In [ ]:
seasons = np.arange(2020,2025)
df1 = get_box("P","Base",False,seasons)
df1["game_id"] = df1["game_id"].astype(int)
df2 = get_box("P","Adv",False,seasons)
df2 = df2[["gameid","personid","possessions"]]
df2.columns = ['game_id','player_id',"poss"]
df3 = pd.merge(df1,df2,on=['game_id','player_id'])
df3["dre"] = -8.42364*df3["poss"]/100+0.79231*df3["pts"]-0.71944*(df3["fga"]-df3["fg3a"])-0.55233*df3["fg3a"]-0.15944*df3["fta"]+0.13479*df3["oreb"]+0.3996*df3["dreb"]+0.54415*df3["ast"]+1.68007*df3["stl"]+0.76387*df3["blk"]-1.3599*df3["tov"]-0.10838*df3["pf"]

df3["gmsc"] = df3["pts"]+0.4*df3["fgm"]-0.7*df3["fga"]-0.4*(df3["fta"]-df3["ftm"])+0.7*df3["oreb"]+0.3*df3["dreb"]+df3["stl"]+0.7*df3["ast"]+0.7*df3["blk"]-0.4*df3["pf"]-df3["tov"]
df3["game_date"] = pd.to_datetime(df3["game_date"])

df3['player_name'] = (
        df3['player_name']
        .str.normalize('NFKD')
        .str.encode('ascii', errors='ignore')
        .str.decode('utf-8')
    )

In [ ]:
df4 = df3[['player_id', 'player_name', 'game_date', 'dre', 'season']]
df4 = df4.sort_values(["game_date"],ascending=True)
df4g = df4.groupby(["season","player_id","player_name"])
keys = list(df4g.groups)
dfa = []
for key in tqdm(keys):
    df5 = df4g.get_group(key)
    df5 = df5.reset_index(drop=True)
    df5 = df5.reset_index()
    df5 = df5.rename(columns={"index":"game_no"})
    df5["game_no"] +=1
    dfa.append(df5)
df6 = pd.concat(dfa)
# df7["Season"] = df7["season"]
df6["Season"] = df6["season"].astype("category")

In [ ]:
player = "Austin Reaves"
df7 = df6.query(f'player_name == "{player}"')
df7 = df7.query(f'Season == 2025')
# player_id = 202710
# df7 = df6.query(f"player_id == {player_id}")
p = (
    ggplot(df7)
    + aes(x="game_no",y="dre", group="Season",color="Season")
    + geom_smooth(se=False,span=0.7,size=2)
    + geom_point(size=1)
    + theme_idv
    + pnba
    + theme(
        plot_subtitle=element_text(size=12),
        plot_title=element_text(size=20),
    )
    + labs(
        title=f"{player} Peformance Trends",
        subtitle="Evaluation metric is DRE (Daily RAPM Estimate) by Kevin Ferrigan",
        y="DRE",
        x="Game #",
    )
)
p

In [ ]:
df7